----
<img src="../../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Delivery - OMM Stream - Market By Price data via callback

This notebook demonstrates how to use the OMM Stream interface to request streaming Full Depth Orderbook data

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the RD Library in the **_refinitiv-data.config.json_** configuration file - as described in the Quick Start -> Sessions example.

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _RD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [2]:
from refinitiv.data.delivery import omm_stream
import refinitiv.data as rd
import datetime
import json

## Open the default session

To open the default session ensure you have a '*refinitiv-data.config.json*' in the ***Configuration*** directory, populated with your credentials and specified a 'default' session in the config file    


In [3]:
rd.open_session()

<refinitiv.data.session.platform.Definition object at 0x1ba9ce49fa0 {session_name='rdp'}>

# Example

## Define a function to display events

In [4]:
# Callback function to display data or status events
def display_event(eventType, event):
    currentTime = datetime.datetime.now().time()
    print("----------------------------------------------------------")
    print(">>> {} event received at {}".format(eventType, currentTime))
    # Print only 1000 characters to limit the output
    print(json.dumps(event, indent=2)[0:1000])
    return

## Create an OMM Stream and register the event callbacks

In [5]:
stream = omm_stream.Definition(name="VOD.L", domain='MarketByPrice').get_stream()

# Refresh - the first full imaage we get back from the server
stream.on_refresh(lambda event, stream : display_event("Refresh", event))
# Update - as and when field values change, we receive updates from the server
stream.on_update(lambda event, stream : display_event("Update", event))
# Status - if data goes stale or item closes, we get a status message
stream.on_status(lambda event, stream : display_event("Status", event))
# Other errors
stream.on_error(lambda event, stream : display_event("Error", event))

## Open the Item Stream
The **open()** call to open the OMM Stream is a synchronous one. This means the first event (either via on_refresh(), on_status() or on_error()) can occur before the open() method returns.   
However, if we use the **open_async()** asynchronous method instead, the first event callback will be after  open_async() returns.

In [6]:
# Library will request OrderBook from server
stream.open()
# We should intially receive the full orderbook, 
# after which we will receive updates for specific order (Add, Update, Delete)
# Note from the above display_event() function that I am dumping just the first 1000 characters to minimise output
# You should remove this limit to see the full response.

----------------------------------------------------------
>>> Refresh event received at 12:31:14.074440
{
  "ID": 1,
  "Type": "Refresh",
  "Domain": "MarketByPrice",
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "VOD.L"
  },
  "State": {
    "Stream": "Open",
    "Data": "Ok",
    "Text": "**All is well"
  },
  "Complete": false,
  "Qos": {
    "Timeliness": "Realtime",
    "Rate": "JitConflated"
  },
  "PermData": "AwEBJJw=",
  "SeqNumber": 40736,
  "Map": {
    "KeyType": "Buffer",
    "Summary": {
      "Fields": {
        "PROD_PERM": 249,
        "DSPLY_NAME": "VODAFONE GROUP",
        "CURRENCY": "GBp",
        "ACTIV_DATE": "2021-11-26",
        "LOT_SIZE_A": 1,
        "RECORDTYPE": 113,
        "SEQNUM": 16976099,
        "RDN_EXCHD2": "LSE",
        "PROV_SYMB": "133215",
        "PR_RNK_RUL": "NOR",
        "OR_RNK_RUL": "PT  ",
        "MNEMONIC": "VOD",
        "PERIOD_CDE": "T",
        "TRD_TYPE": null,
        "TIMACT_MS": 45070891,
        "CONTEXT_ID": 2105,


<OpenState.Opened: 1>

----------------------------------------------------------
>>> Update event received at 12:31:14.580127
{
  "ID": 1,
  "Type": "Update",
  "Domain": "MarketByPrice",
  "UpdateType": "Unspecified",
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "VOD.L"
  },
  "SeqNumber": 40752,
  "Map": {
    "KeyType": "Buffer",
    "Summary": {
      "Fields": {
        "TIMACT_MS": 45073867,
        "SEQNUM": 16976699
      }
    },
    "Entries": [
      {
        "Action": "Update",
        "Key": "MTExLjE4QQ==",
        "Fields": {
          "ORDER_PRC": 111.18,
          "ORDER_SIDE": "ASK",
          "ACC_SIZE": 11586,
          "NO_ORD": 5,
          "LV_TIM_MS": 45073867
        }
      }
    ]
  }
}
----------------------------------------------------------
>>> Update event received at 12:31:14.921960
{
  "ID": 1,
  "Type": "Update",
  "Domain": "MarketByPrice",
  "UpdateType": "Unspecified",
  "Key": {
    "Service": "ELEKTRON_DD",
    "Name": "VOD.L"
  },
  "SeqNumber": 40880,
  "Map

You may notice that there are multiple Refresh events - this is often the case for the more actively traded instruments with large orders books.
Once all Refresh events have been received, you can then expect to receive Update events with just the Order changes i.e. Add, Update or Delete orders.  
**NOTE:** I am truncating the output to the 1st 1000 character of each response payload - for ease of viewing.  


## Close Stream

In [7]:
stream.close()

<OpenState.Closed: 3>

### Close Session

In [8]:
rd.close_session()